<h1>PCS Data Science Workshop</h1>

Welcome to the PCS Data Science Workshop! This notebook will provide a brief overview of numerous data science techniques, many of which is covered in Data C100 (highly recommend) including
* `Python`
* `Numpy` and Vector Operations
* `pandas`, EDA, and Data Manipulation
* Visualization (`Matplotlib/Seaborn`)
* Multilinear Modeling, Data Engineering, Gradient Descent, and `Scikit-Learn`

## Table of Contents
* [**Python**](#python)
    * [Lambda Functions](#lambda)
    * [Iteration](#iteration)
    * [List Comprehensions](#lc)
* [**Vectorized Operations**](#vo)
    * [How They Work](#how-vo-works)
    * [Time Comparison (Iterated vs. Vectorized Operations)](#time-comp)
* [**pandas and DataFrames**](#pd-df)
    * [Accessing Rows, Columns, and Subsets](#df-slicing)
    * [Filtering](#filtering)
    * [Exploratory Data Analysis (EDA)](#eda)
        * [You do some EDA](#your-eda)
    * [Handling NaN and Missing Values](#nan)
    * [Merges and Joins](#merge)
* [**Visualization**](#visualization)

#### First, Let's do Some Top Level Imports

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.graph_objects
import seaborn as sns
import matplotlib.pyplot as plt

# Data Training
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Other things you may need
import os
import asyncio
import aiohttp
import time

# Turning warnings off
import warnings
warnings.simplefilter("ignore")

<h1 id="python"><a class="anchor" id="python">Python</a></h1>

Python is the (arguably) the most important language used for data science, and most of the low level work you do will be in the Jupyter environment (which was developed by Cal's own professor, Fernando Perez!)

Here are some important concepts to know

<h3><a class="anchor" id="lambda">Lambda Functions</a></h3>

In [ ]:
# A function which returns x squared
lambda x: x ** 2

# A function which determines if a string has the word "at"
lambda x: "at" in x

# A function which finds if the percent of people who have a disability is greater than 20%
lambda x: (x["Disability"] / x["Total Population"]) > 0.2;

<h3><a class="anchor" id="iteration">Iteration</a></h3>

In [ ]:
lst1 = ["hello", "this", "is"]
lst2 = ["a", "great", "workshop"]

# A for loop counter
for i in range(len(lst1)):
    print(lst1[i])

# A for each loop
for word in lst2:
    print(word)

<h3><a class="anchor" id="lc">List Comprehensions</a></h3>

In [ ]:
# Checks if the words in list 1 and 2 have a length greater than 4
[len(word) > 4 for word in lst1 + lst2]

<h1><a class="anchor" id="vo">Vectorized Operations</a></h1>

In [ ]:
def current_time():
    return int(time.time() * 1000)  # Get current time in milliseconds

When manipulating large amounts of data, it becomes slower and slower to iterate through every value to transform it... let's do a test

In [ ]:
for length in [10, 1000, 100000, 1000000, 10000000]:
    t = current_time()
    result = [x ** 2 + 1 for x in range(length)]
    elapsed_time = current_time() - t
    print(f"Iterating through {length} values took {elapsed_time} milliseconds")

To solve this, we can "vectorize" our operations using libraries like `Numpy` and `pandas`, which takes advantage of parallelism/multithreading to efficiently apply operations to large quantities of data

In [ ]:
for length in [10, 1000, 100000, 1000000, 10000000]:
    t = current_time()
    result = np.arange(length) ** 2
    elapsed_time = current_time() - t
    print(f"Iterating through {length} values took {elapsed_time} milliseconds")

<h3><a class="anchor" id="time-comp">Time Comparison (Iterated vs. Vectorized Operations)</a></h3>

As you can see, vectorized operations may be slower at first, but are significantly more efficienct when operating on millions of values

In [ ]:
# Create an empty DataFrame to store the times
vectorized_vs_iterated = pd.DataFrame({"Log (length)": [], "Vectorized": [], "Iterated": []})

for length in np.arange(25):
    
    # Vectorized Operation Time
    t = current_time()
    vectorized_result = np.arange(2 ** length) ** 2 + 5
    vectorized_time = current_time() - t

    # Iterated Operation Time
    t = current_time()
    iterated_result = [x ** 2 + 5 for x in range(2 ** length)]
    iterated_time = current_time() - t

    vectorized_vs_iterated = vectorized_vs_iterated.append({"Log (length)": length, "Vectorized": vectorized_time, "Iterated": iterated_time}, ignore_index=True)

# Create a line plot using Plotly
fig = px.line(vectorized_vs_iterated, x="Log (length)", y=["Vectorized", "Iterated"], labels={"Log (length)": "Log(length)", "value": "Time (ms)"}, title="Vectorized vs. Iterated Operation Time Comparison")
fig.show()

<h3><a class="anchor" id="how-vo-works">How Vectorized Operations Actually Work</a></h3>

Using `Numpy`, we can create an `array` and apply functions directly to them like a string or number. In `pandas`, a `Series` will do the trick

In [ ]:
# Using iteration
python_list = range(5)
print(f"Python List Squared: {[x ** 2 for x in python_list]}")

# Using numpy arrays
numpy_array = np.arange(5)
# (Alternatively, you could run numpy_array = np.array(python_list) to turn a list into an array
print(f"Numpy Array Squared: {numpy_array ** 2}")

# Using pandas Series
pandas_series = pd.Series(range(5))
print(f"Pandas Series Squared: \n{pandas_series ** 2}");

# As you can see, both the series and the array have vectorized operations

If we want to slice a list of strings

In [ ]:
pandas_series_str = pd.Series(["Hello", "I", "Will", "Be", "Sliced"])
pandas_series_str.str[0]

If we want to find the ones with length greater than 5

In [ ]:
pandas_series_str.str.len() > 5

This will be very useful for later when it comes to EDA

If you don't know how to do an operation, look at the documentation for these types of things:

[pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)

[pandas.Series.str](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html)

[Numpy.Array](https://numpy.org/doc/stable/reference/arrays.html)

<h1><a class="anchor" id="pd-df">pandas and DataFrames</a></h1>

Here, we load a dataset containing information about all registered electric vehicles in Washington state

Usually, data like this is stored in a `.csv` file, and can be read by `pandas` using `pd.read_csv(f'{route}')`

This turns it into a table, which we call a `DataFrame`

In [ ]:
ev_data = pd.read_csv('./Electric_Vehicle_Population_Data.csv')
ev_data

There could be a few things we want to do with this data... but in particular, we will take the time to answer the following questions:
* How many of each "make" is there?
* How many of each specific "make" and "model" are there?
* How do we handle missing data?
* How does the "Electric Range" compare for different "makes"

<h3><a class="anchor" id="df-slicing">Accessing Rows, Columns, and Subsets</a></h3>

#### The Slicing Operator

The most basic and widely used operator for DataFrames is the slicing operator: `[]`

There are a few things we can do with this:

##### Columns

* Get a column as a Series: `df['column_name']`
* Get a column as a DataFrame: `df[['column_name']]`
* Get multiple columns: `df[['column_name_1', 'column_name_2']]`

##### Rows

* Slice from m to n rows: `df[m:n]`
* Get only rows with a `True` value (**important**): `df[[True, False, True, True... False, True]]`

Let's look at how we can use these operations

In [ ]:
# Getting a series
ev_data["Make"]

In [ ]:
# Getting it as a DataFrame instead
ev_data[["Make"]] # You can think of this as a 2 dimensional object instead of 1 (we add another set of brackets)

We can apply vectorized operations to Series objects

In [ ]:
KILOMETERS_PER_MILE = 1.60934
ev_data["Electric Range"] * KILOMETERS_PER_MILE

We can even use this to add a column to our DataFrame

In [ ]:
ev_data["Electric Range (km)"] = ev_data["Electric Range"] * KILOMETERS_PER_MILE
ev_data

If I want to get the `"Make"`, `"Model"`, and `"Electric Range"` columns

In [ ]:
ev_data[["Make", "Model", "Electric Range"]] # Notice the double brackets

Single brackets here error out because it thinks we are trying to access rows

In [ ]:
try:
    ev_data["Make", "Model", "Electric Range"]
except KeyError as k:
    print(f"{k} cannot be accessed as rows in this DataFrame")

Accessing the first five rows

Alternatively, we could run `ev_data.head(5)`

In [ ]:
ev_data[:5]

<h3><a class="anchor" id="filtering">Filtering (<b>important</b>)</a></h3>

What if we want to sort to only where the `"Make"` is "TESLA"?

We can start by accessing the `"Make"` column

In [ ]:
make = ev_data["Make"]
make

We can then use **vectorized operations** to find where the `"Make"` is "TESLA"

In [ ]:
make_is_tesla = (make == "TESLA")
make_is_tesla

We can then use this to access the DataFrame only where the `"Make"` is "TESLA"

Remember this...? 

*Get only rows with a `True` value (**important**): `df[[True, False, True, True... False, True]]`*

In [ ]:
ev_data[make_is_tesla]

All in all, we can run it in one operation as so, which is the standard for non-complex operations 

A good way of thinking about this is:

*ev_data where ev_data column "Make" is "TESLA"*

In [ ]:
ev_data[ev_data["Make"] == "TESLA"]

Let's do another where we find the rows where the `"Electric Range"` is greater than 200

In [ ]:
ev_data[ev_data["Electric Range"] > 200]

<h3><a class="anchor" id="eda">Exploratory Data Analysis (EDA)</a></h3>

By far, the most important topic covered today! This is just a brief overview, but we will show you how to do some interesting EDA on the dataset, and introduce the idea of data cleaning

Here are some of the functions we will look at:
* `df.value_counts("column")` - counting values
* `df.sort_values("column")` - sorting by a column
* `df.groupby("column").agg(func)` - grouping by a column (or multiple) and aggregating by a function
* `df.groupby("column").filter(func)` - groupbing by a column (or multiple) and filtering by a function

In [ ]:
ev_data.value_counts("Make")

In [ ]:
ev_data.sort_values("Electric Range", ascending=False)

In [ ]:
ev_data.value_counts(["Make", "Model"])

If we want to find the average `"Electric Range"` per vehicle `"Make"`

In [ ]:
ev_data[["Make", "Electric Range"]].groupby("Make").agg(np.mean).sort_values("Electric Range", ascending=False)

If we want to filter to rows where there are at least 500 of that car `"Make"`

In [ ]:
at_least_500 = ev_data.groupby("Make").filter(lambda x: len(x) >= 500)
at_least_500

In [ ]:
# Let's do a sanity check!
at_least_500.value_counts("Make")

<h3><a class="anchor" id="your-eda">You do some EDA!!!</a></h3>

Give us some things to look at for this DataFrame :) - we will show you how to do it!

In [ ]:
# EDA

In [ ]:
# EDA

In [ ]:
# EDA

In [ ]:
# EDA

In [ ]:
# EDA

<h3><a class="anchor" id="nan">Handling NaN or Missing Values</a></h3>

In [ ]:
at_least_500[at_least_500["Legislative District"].isna()]

In [ ]:
non_wa = at_least_500[at_least_500["State"] != "WA"]
non_wa

In [ ]:
all(non_wa["Legislative District"].isna())

Looks like we found that the "Washington" data contains information from a few other states and British Columbia... we aren't interested in that data, so we can drop it

In [ ]:
ev_data_wa = at_least_500[at_least_500["State"] == "WA"]
ev_data_wa

<h3><a class="anchor" id="merge">Merges and Joins</a></h3>

We can even use the data from multiple tables to enrich a single `DataFrame` - Let's use the ZIP code to ZCTA excel mapping as an example

What if I want to know what the ZCTA is for each of these areas?

In [ ]:
# Read the file in
zip_to_zcta = pd.read_excel('./ZIP_to_ZCTA.xlsx') # Notice the different extension on this one
zip_to_zcta

It looks like something is wrong with the `"ZIP_CODE"` and `"zcta"` columns... let's clean them

In [ ]:
clean_zip = lambda x: ((5 - x.astype(str).str.len()).apply(lambda y: y * "0") + x.astype(str))
clean_zcta = lambda x: ((7 - x.astype(str).str.len()).apply(lambda y: y * "0") + x.astype(str).str[:-2])
zip_to_zcta["ZIP_cleaned"] = clean_zip(zip_to_zcta["ZIP_CODE"])
zip_to_zcta["ZCTA_cleaned"] = clean_zcta(zip_to_zcta["zcta"])
zip_to_zcta

Let's also clean the zip in `ev_data_wa`

In [ ]:
ev_data_wa["ZIP"] = clean_zcta(ev_data_wa["Postal Code"])
ev_data_wa

That's better! Now let's perform a merge using `df.merge` [(documentation)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

In [ ]:
ev_data_with_zcta = ev_data_wa.merge(zip_to_zcta[["ZIP_cleaned", "ZCTA_cleaned"]],
                                     left_on="ZIP",
                                     right_on="ZIP_cleaned")
ev_data_with_zcta.drop(columns=["Postal Code"]) # We can drop the 'Postal Code' column
ev_data_with_zcta

Perfect! What we did represents most of the EDA, data cleaning, and other things you may need to do with `pandas` - it's time to move on to...

<h1><a class="anchor" id="visualization">Visualization</a></h1>

For visualization, we will be using `matplotlib.pyplot` (`plt`) and `seaborn` (`sns`), but `plotly` is also a great option!